# Phenotype/SNP relation extraction from tables

In [38]:
%load_ext autoreload
%autoreload 2

import sys
import cPickle

# import snorkel and gwasdb
sys.path.append('../snorkel')
sys.path.append('../src')
sys.path.append('../src/crawler')

# set up paths
abstract_dir = '../data/db/papers'

# set up matplotlib
import numpy as np
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,4)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load corpus

In [20]:
from snorkel.parser import XMLDocParser
from extractor.parser import UnicodeXMLTableDocParser

xml_parser = UnicodeXMLTableDocParser(
    path=abstract_dir,
    doc='./*',
    text='.//table',
    id='.//article-id[@pub-id-type="pmid"]/text()',
    keep_xml_tree=True)

In [21]:
from snorkel.parser import HTMLParser
from extractor.parser import UnicodeTableParser
from snorkel.parser import CorpusParser
import cPickle

table_parser = UnicodeTableParser()
html_parser = HTMLParser(path='../data/db/papers/')

corpus_name = 'gwas-table-corpus.pkl'

try:
    with open(corpus_name,"r") as pkl:
        corpus = cPickle.load(pkl)
except:
    cp = CorpusParser(xml_parser, table_parser, max_docs=15)
    %time corpus = cp.parse_corpus(name='GWAS Corpus')
    # pickling currently doesn't work...
#     with open(corpus_name,"w") as pkl:
#         corpus = cPickle.dump(corpus, pkl)

CPU times: user 56.3 s, sys: 3.34 s, total: 59.6 s
Wall time: 1min 35s


## Candidate extraction

### RSid Extraction

In [224]:
from snorkel.matchers import DictionaryMatch, RegexMatchSpan, Union
from snorkel.candidates import EntityExtractor
from snorkel.candidates import TableNgrams

from db.kb import KnowledgeBase

# Define a candidate space
ngrams = TableNgrams(n_max=1)

# Get a list of all the RSids we know
kb = KnowledgeBase()
rs_ids = kb.get_rsid_candidates()

# Define matchers
dict_rsid_matcher = DictionaryMatch(d=rs_ids, longest_match_only=False)
regx_rsid_matcher = RegexMatchSpan(rgx=r'rs\d+')
rsid_matcher = Union(dict_rsid_matcher, regx_rsid_matcher)

rsid_extractor = EntityExtractor(ngrams, rsid_matcher)
# %time rs_candidates = rsid_extractor.extract(corpus.get_tables(), name='all')

In [225]:
# for cand in rs_candidates[:10]: 
#     print cand
# print "%s candidates extracted" % len(rs_candidates)
# print rs_candidates[0].context
# print rs_candidates[0].context.cell

#### Statistics

In [226]:
from extractor.util import gold_rsid_stats, gold_rsid_precision

gold_set = frozenset( [ (doc.name, rs_id) for doc in corpus.documents for rs_id in kb.rsids_by_pmid(int(doc.name)) ] )
gold_set_rsids = [rs_id for doc_id, rs_id in gold_set]

gold_rsid_stats(rs_candidates, gold_set)

NameError: name 'rs_candidates' is not defined

Interesting: some SNPs seem to be never mentioned (e.g. rs12122100) while others (rs727153) appear only in the text.
Sometimes, it's not picked up for a different, strange reason: see rs13314993.

In [ ]:
cells = rs_candidates[0].context.cell.aligned_cells('row')
[cell.text for cell in cells]

### Phenotypes

In [227]:
from snorkel.matchers import DictionaryMatch, Union, CellNameMatcher, CellDictNameMatcher
from snorkel.candidates import EntityExtractor
from snorkel.candidates import TableNgrams, CellSpace

# Define a candidate space
ngrams = TableNgrams(n_max=9)
cells = CellSpace()

# Create a list of possible words that could denote phenotypes
phen_words = ['trait', 'phenotype']

# Define matchers
# dict_row_matcher = DictionaryMatch(d=phen_words, longest_match_only=False, stemmer='porter')
# cell_row_matcher = CellNameMatcher(row_matcher=dict_row_matcher, cand_space=ngrams)
# dict_col_matcher = DictionaryMatch(d=phen_words, longest_match_only=False, stemmer='porter')
# cell_col_matcher = CellNameMatcher(col_matcher=dict_col_matcher, cand_space=ngrams)
# phen_matcher = Union(cell_row_matcher, cell_col_matcher)
# phen_matcher = CellNameMatcher(col_matcher=dict_col_matcher, cand_space=ngrams)
phen_matcher = CellDictNameMatcher(axis='col', d=phen_words, n_max=3, ignore_case=True)

phen_extractor = EntityExtractor(cells, phen_matcher)
%time phen_candidates = phen_extractor.extract(corpus.get_tables(), name='all')

CPU times: user 27.2 s, sys: 1.11 s, total: 28.3 s
Wall time: 31.4 s


In [228]:
print "%s candidates extracted" % len(phen_candidates)
for cand in phen_candidates[0:10]: 
    print cand.context.document.name, cand.context.table, cand.context.cell
    print unicode(cand)
#     print [span for span in cand.row_ngrams()]
#     print [span for span in cand.col_ngrams()]
#     print
print
print phen_candidates[0].context
print phen_candidates[0].context.document.name, phen_candidates[0].context.table
print phen_candidates[0].context.cell

1851 candidates extracted
17903292 Table('17903292', 0) Cell('17903292', 0, 10, u'Serum Creatinine')
Span("Serum Creatinine", context=None, chars=[0,15], words=[0,1])
17903292 Table('17903292', 0) Cell('17903292', 0, 15, u'Change in serum creatinine')
Span("Change in serum creatinine", context=None, chars=[0,25], words=[0,3])
17903292 Table('17903292', 0) Cell('17903292', 0, 20, u'Glomerular Filtration Rate (GFR)')
Span("Glomerular Filtration Rate (GFR)", context=None, chars=[0,35], words=[0,5])
17903292 Table('17903292', 0) Cell('17903292', 0, 25, u'Chronic Kidney Disease')
Span("Chronic Kidney Disease", context=None, chars=[0,21], words=[0,2])
17903292 Table('17903292', 0) Cell('17903292', 0, 30, u'Cystatin C')
Span("Cystatin C", context=None, chars=[0,9], words=[0,1])
17903292 Table('17903292', 0) Cell('17903292', 0, 35, u'Uric acid')
Span("Uric acid", context=None, chars=[0,8], words=[0,1])
17903292 Table('17903292', 0) Cell('17903292', 0, 40, u'Urinary Albumin Excretion')
Span("Ur

### Relations

In [229]:
from snorkel.candidates import AlignedTableRelationExtractor
relation_extractor = AlignedTableRelationExtractor(rsid_extractor, phen_extractor, axis='row', induced=True)
tables = corpus.get_tables()

# create smaller subsets for evaluation/debugging
easy_tables = [tables[8]]
# hard_tables = [t for t in tables if t.document.name=='17658951']
hard_doc = [d for d in corpus.documents if d.name == '17903293'][0]
hard_tables = [hard_doc.tables[2]]

In [230]:
%time candidates = relation_extractor.extract(tables, name='all')
print "%s relations extracted, e.g." % len(candidates)
for cand in candidates[:10]: 
    print cand

CPU times: user 19min 33s, sys: 17.2 s, total: 19min 51s
Wall time: 20min 28s
1578 relations extracted, e.g.
SpanPair(Span("rs1158167", context=None, chars=[0,8], words=[0,0]), Span("CysC", context=None, chars=[0,3], words=[0,0]))
SpanPair(Span("rs1712790", context=None, chars=[0,8], words=[0,0]), Span("UAE", context=None, chars=[0,2], words=[0,0]))
SpanPair(Span("rs6977660", context=None, chars=[0,8], words=[0,0]), Span("TSH", context=None, chars=[0,2], words=[0,0]))
SpanPair(Span("rs9322817", context=None, chars=[0,8], words=[0,0]), Span("TSH", context=None, chars=[0,2], words=[0,0]))
SpanPair(Span("rs10499559", context=None, chars=[0,9], words=[0,0]), Span("TSH", context=None, chars=[0,2], words=[0,0]))
SpanPair(Span("rs9305354", context=None, chars=[0,8], words=[0,0]), Span("UAE", context=None, chars=[0,2], words=[0,0]))
SpanPair(Span("rs2145231", context=None, chars=[0,8], words=[0,0]), Span("CysC", context=None, chars=[0,3], words=[0,0]))
SpanPair(Span("rs723464", context=None, c

Here, we remove nested candidates

In [231]:
# load existing candidates into a dict
span_dict = { str(span_pair.span1.context) : list() for span_pair in candidates }
for span_pair in candidates:
    span = span_pair.span1
    span_dict[str(span.context)].append( (span.char_start, span.char_end) )

def nested(ivl1, ivl2):
    if ivl1 != ivl2 and ivl2[0] <= ivl1[0] <= ivl1[1] <= ivl2[1]:
        return True
    else:
        return False

new_candidates = list()
for span_pair in candidates:
    span = span_pair.span1
    span_ivl = span.char_start, span.char_end
    span_name = str(span.context)
    if all([not nested(span_ivl, other_ivl) for other_ivl in span_dict[span_name]]):
        new_candidates.append(span_pair)
        
print len(candidates) - len(new_candidates), 'candidates dropped, now we have', len(new_candidates)
# phen_c = new_phen_c

0 candidates dropped, now we have 1578


## Learning the correctness of relations

### Creating a gold set

To create a gold set, we save all extracted relations into a csv file. We annotate it manually, and save the result to a second file. It contains pairs of phenotype and rsid strings; if that file exists, we take these as gold truth.

In [31]:
# store relations to annotate
with open('rels.acroynms.unnanotated.tsv', 'w') as f:
    for span_pair in new_candidates:
        doc_id = span_pair.span0.context.document.name
        table_id = span_pair.span0.context.table.position
        row_num = span_pair.span0.context.cell.row_num
        str1 = span_pair.span0.get_span()
        str2 = span_pair.span1.get_span()
        try:
            f.write('%s\t%s\t%d\t%s\t%s\n' % (doc_id, table_id, row_num, str1, str2))
        except:
            continue

In [232]:
# load annotations
annotations = dict()
with open('rels.acronyms.annotated.txt') as f:
    text = f.read()
    for line in text.split('\r'):
        doc_id, table_id, col_n, rs_id, phen, res = line.strip().split('\t')
        res = 1 if int(res) == 1 else -1
        annotations[(doc_id, table_id, rs_id, phen)] = res

### Classify correct relations

In [233]:
from snorkel.features import TableNgramPairFeaturizer

pkl_f = 'acro_table_feats.pkl'
try:
    with open(pkl_f, 'rb') as f:
        featurizer = cPickle.load(f)
except:
    featurizer = TableNgramPairFeaturizer()
    featurizer.fit_transform(candidates)

Building feature index...
Extracting features...
0/22341
5000/22341
10000/22341
15000/22341
20000/22341


In [234]:
def spair2uid(span_pair):
    doc_id = span_pair.span0.context.document.name
    table_id = str(span_pair.span0.context.table.position)
    str1 = span_pair.span0.get_span()
    str2 = span_pair.span1.get_span()
    return (doc_id, table_id, str1, str2)

# Split into train and test set
training_candidates = []
gold_candidates     = []
gold_labels         = []
n_half = len(candidates)/2
for c in candidates[:n_half]:
    uid = spair2uid(c)
    if uid in annotations:
        gold_candidates.append(c)
        gold_labels.append(annotations[uid])
    else:
        training_candidates.append(c)
training_candidates.extend(candidates[n_half:])
gold_labels = np.array(gold_labels)
print "Training set size: %s" % len(training_candidates)
print "Gold set size: %s" % len(gold_candidates)
print "Positive labels in training set: %s" % len([c for c in training_candidates if annotations.get(spair2uid(c),0)==1])
print "Negative labels in training set: %s" % len([c for c in training_candidates if annotations.get(spair2uid(c),0)==-1])
print "Positive labels in gold set: %s" % len([c for c in gold_candidates if annotations[spair2uid(c)]==1])
print "Negative labels in gold set: %s" % len([c for c in gold_candidates if annotations[spair2uid(c)]==-1])

Training set size: 884
Gold set size: 694
Positive labels in training set: 789
Negative labels in training set: 0
Positive labels in gold set: 555
Negative labels in gold set: 139


In [235]:
# negative LFs
def LF_number(m):
    txt = m.span1.get_span()
    frac_num = len([ch for ch in txt if ch.isdigit()]) / float(len(txt))
    return -1 if frac_num > 0.6 else 0

def LF_bad_phen_mentions(m):
    top_cells = m.span1.context.cell.aligned_cells(axis='col', induced=True)
    top_phrases = [phrase for cell in top_cells for phrase in cell.phrases]
    if not top_phrases: return 0
    matching_phrases = []
    for phrase in top_phrases:
        if any (phen_matcher._f_span(word) for word in phrase.text.split(' ')):
            matching_phrases.append(phrase)
    small_matching_phrases = [phrase for phrase in matching_phrases if len(phrase.text) <= 25]
    return -1 if not small_matching_phrases else 0

LF_tables_neg = [LF_number, LF_bad_phen_mentions]

# positive LFs
def LF_no_neg(m):
    return +1 if not any(LF(m) for LF in LF_tables_neg) else 0

LF_tables_pos = [LF_no_neg]

LF_tables = LF_tables_neg + LF_tables_pos

In [236]:
from snorkel.snorkel import TrainingSet
from snorkel.features import NgramFeaturizer

training_set = TrainingSet(training_candidates, LF_tables, featurizer=TableNgramPairFeaturizer())

Applying LFs...
Featurizing...
Building feature index...
Extracting features...
0/14220
5000/14220
10000/14220
LF Summary Statistics: 3 LFs applied to 884 candidates
------------------------------------------------------------
Coverage (candidates w/ > 0 labels):		100.00%
Overlap (candidates w/ > 1 labels):		0.00%
Conflict (candidates w/ conflicting labels):	0.00%


In [237]:
from snorkel.snorkel import Learner
import snorkel.learning
from snorkel.learning import LogReg

learner = Learner(training_set, model=LogReg())

# Splitting into CV and test set
n_half = len(gold_candidates)/2
test_candidates = gold_candidates[:n_half]
test_labels     = gold_labels[:n_half]
cv_candidates   = gold_candidates[n_half:]
cv_labels       = gold_labels[n_half:]

from snorkel.learning_utils import GridSearch
gs       = GridSearch(learner, ['mu', 'lf_w0'], [[1e-5, 1e-7],[1.0,2.0]])
gs_stats = gs.fit(cv_candidates, cv_labels)

Testing mu = 1.00e-05, lf_w0 = 1.00e+00
Begin training for rate=0.01, mu=1e-05
	Learning epoch = 0	Gradient mag. = 0.027207
	Learning epoch = 250	Gradient mag. = 0.047458
	Learning epoch = 500	Gradient mag. = 0.076673
	Learning epoch = 750	Gradient mag. = 0.119269
Final gradient magnitude for rate=0.01, mu=1e-05: 0.180
Applying LFs...
Featurizing...
Testing mu = 1.00e-05, lf_w0 = 2.00e+00
Begin training for rate=0.01, mu=1e-05
	Learning epoch = 0	Gradient mag. = 0.052007
	Learning epoch = 250	Gradient mag. = 0.082656
	Learning epoch = 500	Gradient mag. = 0.117942
	Learning epoch = 750	Gradient mag. = 0.163135
Final gradient magnitude for rate=0.01, mu=1e-05: 0.210
Testing mu = 1.00e-07, lf_w0 = 1.00e+00
Begin training for rate=0.01, mu=1e-07
	Learning epoch = 0	Gradient mag. = 0.027207
	Learning epoch = 250	Gradient mag. = 0.047555
	Learning epoch = 500	Gradient mag. = 0.076892
	Learning epoch = 750	Gradient mag. = 0.119632
Final gradient magnitude for rate=0.01, mu=1e-07: 0.180
Testin

In [238]:
learner.test_wmv(test_candidates, test_labels)

Applying LFs...
Featurizing...
Test set size:	347
----------------------------------------
Precision:	1.0
Recall:		1.0
F1 Score:	1.0
----------------------------------------
TP: 288 | FP: 0 | TN: 59 | FN: 0


In [239]:
# preds = learner.predict_wmv(candidates)
acronyms = [spair2uid(c) for (c, p) in zip(candidates, preds) if p == 1]
mislabeled_cand = [(c,p, annotations.get(spair2uid(c), None)) for c, p in zip(candidates, preds) if p != annotations.get(spair2uid(c), p)]
for (c,p,g) in mislabeled_cand[:20]:
    print c.span0.context.document.name, p, g
    print c.span0.context    
    print c.span0.get_span(), c.span1.get_span()
    txt = c.span1.get_span()
    top_cells = c.span1.context.cell.aligned_cells(axis='col', induced=True)
    top_phrases = [phrase for cell in top_cells for phrase in cell.phrases]
    print top_phrases
    matching_phrases = []
    for phrase in top_phrases:
        print [(word,phen_matcher._f_span(word)) for word in phrase.text.split(' ')]
        if any (phen_matcher._f_span(word) for word in phrase.text.split(' ')):
            matching_phrases.append(phrase)
            print phrase
#         print phrase, [phen_matcher._f_span(word) for word in phrase.text.split(' ')]
    print [LF(c) for LF in LF_tables]
    print

17903294 -1.0 1
Phrase('17903294', 1, 9, 0, u'rs6683832')
rs6683832 Fibrinogen
[Phrase('17903294', 1, 0, 0, u'Phenotype*'), Phrase('17903294', 1, 16, 0, u'Fibrinogen'), Phrase('17903294', 1, 24, 0, u'Fibrinogen'), Phrase('17903294', 1, 32, 0, u'Fibrinogen'), Phrase('17903294', 1, 40, 0, u'FVII'), Phrase('17903294', 1, 48, 0, u'FVII'), Phrase('17903294', 1, 56, 0, u'FVII'), Phrase('17903294', 1, 64, 0, u'FVII'), Phrase('17903294', 1, 72, 0, u'FVII'), Phrase('17903294', 1, 80, 0, u'FVII'), Phrase('17903294', 1, 88, 0, u'FVII'), Phrase('17903294', 1, 96, 0, u'FVII'), Phrase('17903294', 1, 104, 0, u'FVII'), Phrase('17903294', 1, 112, 0, u'FVII'), Phrase('17903294', 1, 120, 0, u'PAI1'), Phrase('17903294', 1, 128, 0, u'PAI1'), Phrase('17903294', 1, 136, 0, u'tPA'), Phrase('17903294', 1, 144, 0, u'tPA'), Phrase('17903294', 1, 152, 0, u'tPA'), Phrase('17903294', 1, 160, 0, u'tPA'), Phrase('17903294', 1, 168, 0, u'vWF'), Phrase('17903294', 1, 176, 0, u'vWF'), Phrase('17903294', 1, 184, 0, u'vWF

Save the results

In [240]:
preds = learner.predict_wmv(candidates)
rels = [(c.span0.context.document.name, c.span0.get_span(), c.span1.get_span()) for (c, p) in zip(candidates, preds) if p == 1]
print len(rels), 'relations extracted, e.g.:'
print rels[:10]

# store relations to annotate
with open('rels.acronyms.extracted.tsv', 'w') as f:
    for doc_id, str1, str2 in rels:
        try:
            out = u'{}\t{}\t{}\n'.format(doc_id, unicode(str1), str2)
            f.write(out.encode("UTF-8"))
        except:
            print 'Error in saving:', str1, str2

Applying LFs...
Featurizing...
1374 relations extracted, e.g.:
[('17903292', u'rs1158167', u'CysC'), ('17903292', u'rs1712790', u'UAE'), ('17903292', u'rs6977660', u'TSH'), ('17903292', u'rs9322817', u'TSH'), ('17903292', u'rs10499559', u'TSH'), ('17903292', u'rs9305354', u'UAE'), ('17903292', u'rs2145231', u'CysC'), ('17903292', u'rs723464', u'UAE'), ('17903292', u'rs2113379', u'UAE'), ('17903292', u'rs2839235', u'GFR')]


## Resolve acronyms based on ones extracted earlier

In [241]:
from extractor.dictionary import Dictionary, unravel

D = Dictionary()
D.load('acronyms.extracted.tsv')
print len(D), 'definitions loaded'

223 definitions loaded


Use dictionary to resolve acronyms

In [242]:
new_rels = [ (doc_id, rs_id, unravel(doc_id, phen, D)) for doc_id, rs_id, phen in rels ]

In [248]:
for rel in new_rels:
    if rel[0] == '17658951':
        print rel

In [190]:
print D.storage['17903292']
print D.evidence['17903292']

{'ckd': set(['chronic kidney disease']), 'fbat': set(['family-based association tests', 'mean p-value']), 'snp': set(['single nucleotide polymorphism']), 'tsh': set(['thyroid stimulating hormone', 'thyroid stimulation hormone']), 'mb': set(['physical location']), 'gfr': set(['glomerular filtration rate']), 'lh': set(['luteinizing hormone']), 'fsh': set(['follicle stimulating hormone']), 'cysc': set(['cystatin-c']), 'gee': set(['p_value', 'mean p-value', 'generalized estimating equations']), 'dheas': set(['dehydroepiandrosterone sulfate']), 'uae': set(['urinary albumin excretion'])}
{('ckd', 'chronic kidney disease'): 1, ('tsh', 'thyroid stimulating hormone'): 2, ('gfr', 'glomerular filtration rate'): 4, ('fbat', 'family-based association tests'): 1, ('fsh', 'follicle stimulating hormone'): 1, ('lh', 'luteinizing hormone'): 1, ('fbat', 'mean p-value'): 2, ('mb', 'physical location'): 6, ('dheas', 'dehydroepiandrosterone sulfate'): 2, ('gee', 'generalized estimating equations'): 1, ('uae

In [191]:
unravel('17903292', u'CysC', D)

'cystatin-c'

## Evaluate extracted relations

Let's first evaluate the recall w.r.t. GWAS Central

In [247]:
for doc in corpus.documents:
    assocs = [assoc for pmid in pmids for assoc in kb.assoc_by_pmid(doc.name) if assoc.source == 'gwas_central' and assoc.pvalue < 1e-5]
    print doc.name, len(assocs), len([(pmid, rsid, phen) for pmid, rsid, phen in new_rels if pmid == doc.name])
    

17447842 264 0
17658951 312 0
17684544 384 0
17903292 108 90
17903293 840 79
17903294 552 100
17903295 456 174
17903296 840 141
17903297 624 108
17903298 96 128
17903300 288 52
17903301 480 177
17903302 672 111
17903303 408 111
17903304 132 103


In [307]:
pmids = sorted(list({pmid for pmid, _, _ in new_rels}))

from db.kb import KnowledgeBase
kb = KnowledgeBase()
assocs = [assoc for pmid in pmids for assoc in kb.assoc_by_pmid(pmid) if assoc.source == 'gwas_central' and assoc.pvalue < 1e-5]
print len(pmids), len(assocs)

12 446


In [305]:
print pmids

['17903292', '17903293', '17903294', '17903295', '17903296']


In [245]:
# collect resolved relations
rel_dict = { (pmid, rsid) : set() for (pmid, rsid, phen) in new_rels }
for (pmid, rsid, phen) in new_rels:
    rel_dict[(pmid, rsid)].add(phen)

gold_rel_dict = { (a.paper.pubmed_id, a.snp.rs_id) : set() for a in assocs }
for a in assocs:
    gold_rel_dict[(a.paper.pubmed_id, a.snp.rs_id)].add(a.phenotype.name)

First, evaluate recall: how many associations in GWAS central can we recover?

In [249]:
for a in assocs[:50]:
    s1 = gold_rel_dict[(a.paper.pubmed_id, a.snp.rs_id)]
    s2 = rel_dict.get((str(a.paper.pubmed_id), a.snp.rs_id), {})
    if len(s1) != 1 or len(s2) != 1:
        print a.paper.pubmed_id, a.snp.rs_id, a.source
        print 'GWC:', gold_rel_dict[(a.paper.pubmed_id, a.snp.rs_id)]
        print 'US: ', rel_dict.get((str(a.paper.pubmed_id), a.snp.rs_id), None)
        print

Second question: can we learn any more SNPs than the ones that are already in GWAS central?

In [250]:
pmids = sorted(list({pmid for pmid, _, _ in new_rels if int(pmid) < 17903297}))

from db.kb import KnowledgeBase
kb = KnowledgeBase()
assocs = [assoc for pmid in pmids for assoc in kb.assoc_by_pmid(pmid) if assoc.source == 'gwas_central']
print len(assocs)

233


In [252]:
for a in assocs:
    s1 = gold_rel_dict[(a.paper.pubmed_id, a.snp.rs_id)]
    s2 = rel_dict.get((str(a.paper.pubmed_id), a.snp.rs_id), {})
    print a.paper.pubmed_id, a.snp.rs_id, a.source
    print 'GWC:', gold_rel_dict[(a.paper.pubmed_id, a.snp.rs_id)]
    print 'US: ', rel_dict.get((str(a.paper.pubmed_id), a.snp.rs_id), None)
    print

17903292 rs1712790 gwas_central
GWC: set([u'Urinary albumin excretion'])
US:  set(['urinary albumin excretion'])

17903292 rs1243400 gwas_central
GWC: set([u'Urinary albumin excretion'])
US:  set(['urinary albumin excretion'])

17903292 rs9305354 gwas_central
GWC: set([u'Urinary albumin excretion'])
US:  set(['urinary albumin excretion'])

17903292 rs723464 gwas_central
GWC: set([u'Urinary albumin excretion'])
US:  set(['urinary albumin excretion'])

17903292 rs10485409 gwas_central
GWC: set([u'Urinary albumin excretion'])
US:  set(['urinary albumin excretion'])

17903292 rs6977660 gwas_central
GWC: set([u'Thyroid stimulating hormone'])
US:  set(['thyroid stimulating hormone'])

17903292 rs9322817 gwas_central
GWC: set([u'Thyroid stimulating hormone'])
US:  set(['thyroid stimulating hormone'])

17903292 rs10499559 gwas_central
GWC: set([u'Thyroid stimulating hormone'])
US:  set(['thyroid stimulating hormone'])

17903292 rs1158167 gwas_central
GWC: set([u'Cystatin C'])
US:  set(['cystat

## Combine with extracted pvalue/rsid relations

In [347]:
pval_rsid_dict = dict()
pval_dict = dict() # combine all of the pvalues for a SNPs in the same document into one set
with open('pval-rsid.raw.tsv') as f:
    for line in f:
        pmid, rsid, table_id, row_id, col_id, pval = line.strip().split('\t')
        pval, table_id, row_id, col_id = float(pval), int(table_id), int(row_id), int(col_id)
        
        if pmid not in pval_rsid_dict: pval_rsid_dict[pmid] = dict()
        key = (rsid, table_id, row_id)
        if key not in pval_rsid_dict[pmid]: pval_rsid_dict[pmid][key] = set()
        pval_rsid_dict[pmid][key].add(pval)
                
        if pmid not in pval_dict: pval_dict[pmid] = dict()
        if rsid not in pval_dict[pmid]: pval_dict[pmid][rsid] = set()
        pval_dict[pmid][rsid].add(pval)

pval_dict0 = {pmid : {rsid : min(pval_dict[pmid][rsid]) for rsid in pval_dict[pmid]} for pmid in pval_dict}
pval_rsid_dict0 = {pmid : {key : min(pval_rsid_dict[pmid][key]) for key in pval_rsid_dict[pmid]} for pmid in pval_rsid_dict}
pval_dict = pval_dict0
pval_rsid_dict = pval_rsid_dict0

Plan. If phen/rsid has been extracted from tables: take its pvalue from pval_rsid_dict.

If not, we assume that paper has only one phenotype and we take the smallest reported pvalue in the paper.

Our goal for now is just to filter phen/rsid relations that have pval<1e-5.

#### Save all relations that are sufficiently small p-values

In [348]:
# preds = learner.predict_wmv(candidates)
# predicted_candidates = [c for (c, p) in zip(candidates, preds) if p == 1]

import re
import unicodedata
def _normalize_str(s):
    try:
        s = s.encode('utf-8')
        return s
    except UnicodeEncodeError: 
        pass
    try:
        s = s.decode('utf-8')
        return s
    except UnicodeDecodeError: 
        pass    
    raise Exception()

with open('phen-rsid.table.rel.tsv', 'w') as f:
    for c in predicted_candidates:
        pmid = c.span0.context.document.name
        rsid = c.span0.get_span()
        phen = c.span1.get_span()        
        table_id = c.span0.context.table.position
        row_num = c.span0.context.cell.row_num
        col_num = c.span0.context.cell.col_num # of the rsid

        phen = (unravel(pmid, phen, D))
        if isinstance(phen, unicode):
            phen = phen.encode('utf-8')
                    
        pval = pval_rsid_dict[pmid].get((rsid, table_id, row_num), -1)
        if pval > 1e-5: continue

        out_str = '{pmid}\t{rsid}\t{phen}\t{pval}\ttable\t{table_id}\t{row}\t{col}\n'.format(
                    pmid=pmid, rsid=rsid, phen=phen, pval=pval, table_id=table_id, row=row_num, col=col_num)
        f.write(out_str)

In [353]:
print [(c, c.span0.context.cell.row_num, unravel(c.span0.context.document.name, c.span1.get_span(), D)) for c in candidates if c.span0.get_span() == 'rs10500631']

[]


In [352]:
pval_rsid_dict['17903294'].get(('rs10500631', 1, 5), -1)

-1

In [351]:
for x in pval_rsid_dict['17903294']:    
    print x, pval_rsid_dict['17903294'][x]

('rs565229', 2, 12) 4e-06
('rs4591494', 1, 10) 9e-06
('rs7844723', 2, 5) 2e-06
('rs7319671', 1, 25) 3.6e-05
('rs1001254', 1, 9) 2e-05
('rs1245072', 1, 6) 1.8e-05
('rs4460176', 1, 16) 3e-06
('rs1359339', 2, 23) 2.2e-05
('rs1561478', 5, 6) 0.00016
('rs10502583', 2, 16) 1.1e-05
('rs6683832', 1, 1) 5.6e-05
('rs10488360', 1, 11) 7e-06
('rs1510107', 2, 2) 1e-05
('rs953377', 1, 20) 1.5e-05
('rs1649053', 5, 8) 0.0001
('rs10485968', 2, 24) 1.6e-05
('rs6811964', 2, 14) 1.3e-05
('rs966321', 1, 5) 8e-06
('rs4861952', 5, 3) 3.5e-05
('rs848523', 2, 11) 2.1e-05
('rs708356', 1, 23) 1.5e-05
('rs10494076', 1, 18) 3.1e-05
('rs6108011', 2, 21) 6e-06
('rs2175271', 5, 9) 0.00031
('rs9295740', 1, 21) 2.8e-05
('rs7956194', 4, 10) 0.0002
('rs1423741', 5, 11) 0.00076
('rs561241', 6, 1) 0.0
('rs727435', 1, 12) 1e-05
('rs1582055', 7, 4) 7.7e-05
('rs1829883', 2, 6) 6e-06
('rs1958208', 2, 15) 1.1e-05
('rs10484128', 2, 10) 6e-06
('rs1954971', 1, 24) 1e-05
('rs4133289', 2, 1) 0.0
('rs1200821', 2, 7) 6e-06
('rs9253', 